In [13]:
import math
import random
from heapq import merge

import plotly.graph_objects as go

def load_data(file_name: str) -> tuple[list[tuple[int, int]], int]:
    """
    Loads knapsack data from a file.

    Parameters:
    - file_name (str): The file containing item weights and values.

    Returns:
    - tuple: A list of (weight, value) pairs and the maximum weight.
    """
    weights_and_values = []
    with open(file_name) as f:
        lines = f.readlines()
        for line in lines[1:-1]:  # Ignore first and last line
            parts = line.split()
            weights_and_values.append((int(parts[2]), int(parts[1])))  # (weight, value)

        max_weight = int(lines[-1].strip())  # Extract max weight
    return weights_and_values, max_weight


def generate_random_solution(n, max_weight, items) -> list[int]:
    weight = max_weight + 1
    solution = []
    while weight > max_weight:
        solution = [random.randint(0, 1) for _ in range(n)]
        weight = sum(solution[i] * items[i][0] for i in range(n))
    return solution

def calculate_weight(solution: list[int], items: list[tuple[int, int]]) -> int:
    return sum(solution[i] * items[i][0] for i in range(len(items)))

def calculate_value(solution: list[int], items: list[tuple[int, int]]) -> int:
    return sum(solution[i] * items[i][1] for i in range(len(items)))






In [14]:
def generate_crossover(candidate1, candidate2):
    if len(candidate1) != len(candidate2):
        raise ValueError("Candidates must be of the same length")

    length = len(candidate1)
    mask = random.sample(range(length), length // 2)

    child = [None] * length
    for i in mask:
        child[i] = candidate1[i]

    for i in range(length):
        if child[i] is None:
            child[i] = candidate2[i]

    return child

def mutate(candidate, mutation_rate=0.05):
    for i in range(len(candidate)):
        if random.random() < mutation_rate:
            candidate[i] = 1 - candidate[i]
    return candidate

def evaluate_solution(solution, items, max_weight):
    total_weight = sum(solution[i] * items[i][0] for i in range(len(items)))
    if total_weight > max_weight:
        return 0
    return sum(solution[i] * items[i][1] for i in range(len(items)))

def generate_next_generation(candidates, items, max_weight, mutation_rate=0.1):
    next_generation = []
    candidates.sort(key=lambda c: evaluate_solution(c, items, max_weight), reverse=True)
    while len(next_generation) < len(candidates):
        # pick random candidates from top 50%
        candidate1, candidate2 = random.choices(candidates[:len(candidates)//2], k=2)
        child = generate_crossover(candidate1, candidate2)
        child = mutate(child, mutation_rate)
        next_generation.append(child)
    return next_generation

In [15]:
def genetic_algorithm_knapsack(items, max_weight, population_size=50, generations=100):
    population = [[random.randint(0, 1) for _ in range(len(items))] for _ in range(population_size)]
    best_solution = []
    best_value = math.inf
    for _ in range(generations):
        population = generate_next_generation(population, items, max_weight)
        best_solution = max(population, key=lambda c: evaluate_solution(c, items, max_weight))
        best_value = evaluate_solution(best_solution, items, max_weight)

    return best_solution, best_value


In [16]:
items200, max_weight200 = load_data("rucsac-200.txt")
items20, max_weight20 = load_data("knapsack-20.txt")


solution1, value1 = genetic_algorithm_knapsack(items200, max_weight200, 500, 10)
solution2, value2 = genetic_algorithm_knapsack(items200, max_weight200, 2000, 50)
solution3, value3 = genetic_algorithm_knapsack(items20, max_weight20, 150, 5)
solution4, value4 = genetic_algorithm_knapsack(items20, max_weight20, 150, 10)

print("Best Solution knapsack 200 for 100 iterations:", solution1, "Best Value:", value1)
print("Best Solution knapsack 200 for 1000 iterations:", solution2, "Best Value:", value2)
print("Best Solution knapsac 20 for 100 iterations:", solution3, "Best Value:", value3)

Best Solution knapsack 200 for 100 iterations: [1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1] Best Value: 132649
Best Solution knapsack 200 for 1000 iterations: [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,